### Business Problem
An e-commerce company wants to segment its customers and determine marketing strategies according to these segments.

For this purpose, we will define the behavior of customers and we will form groups according to clustering.

### Column Descriptors
InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. <br>
StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product. <br>
Description: Product (item) name. Nominal. <br>
Quantity: The quantities of each product (item) per transaction. Numeric. <br>
InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated. <br>
UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£). <br>
CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer. <br>
Country: Country name. Nominal. The name of the country where a customer resides. <br>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")

In [11]:
# load dataset

df = pd.read_csv('C:/Users/vumac/Desktop/Springboard_Capstone3/Dataset/online_retail_cleaned.csv', encoding= 'unicode_escape', index_col=0)

In [12]:
# Create a 'Revenue' column

df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df["Revenue"] = df["Quantity"]*df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


In [13]:
# Check the lastest invoice date

df.InvoiceDate.max()

Timestamp('2011-12-09 00:00:00')

In [14]:
# Do not know when the dataset was extracted.
# Set current day to 3 days from the latest invoice, to ensure no frequency value is 0

today_date = df['InvoiceDate'].max() + dt.timedelta(days=3)
today_date

Timestamp('2011-12-12 00:00:00')

In [15]:
# Calculate the RFM metrics:
# Recency: The age of the customer at the time of their last purchase.
# Monetary: The total or average sales of the customer.
# Frequency: Number of purchases/transactions.
# Age: The age of the customer since the date of a customer's first purchase to the current date.

rfm = df.groupby("Customer ID").agg({"InvoiceDate": [lambda x: (today_date - x.max()).days,
                                                     lambda x: (today_date - x.min()).days],
                               "Invoice": lambda x: x.nunique(),
                               "Revenue": lambda x: x.sum()})

rfm.columns = ["recency", "age", "frequency", "monetary"]

rfm = rfm[["recency", "frequency", "monetary", "age"]]

rfm.head()

,recency,frequency,monetary,age
Customer ID,,,,
12347.0,5,7,4060.40,370
12348.0,78,4,1186.68,361
12349.0,21,1,1353.80,21
12350.0,313,1,294.40,313
12352.0,39,7,1385.74,299


In [16]:
# Give a score from 1 to 5 to each value for ranking

rfm["R"] = pd.qcut(rfm['recency'], q=5, labels=range(5, 0, -1))

rfm["F"] = pd.qcut(rfm['frequency'].rank(method="first"), q=5, labels=range(1, 6))

rfm["M"] = (pd.qcut(rfm['monetary'], q=5, labels=range(1, 6)))

rfm.head()

,recency,frequency,monetary,age,R,F,M
Customer ID,,,,,,,
12347.0,5,7,4060.40,370,5,5,5
12348.0,78,4,1186.68,361,2,4,4
12349.0,21,1,1353.80,21,4,1,4
12350.0,313,1,294.40,313,1,1,2
12352.0,39,7,1385.74,299,3,5,4


In [17]:
# Concatenate RFM score values to RFM_Segment

def join_rfm(x): return str(int(x['R'])) + str(int(x['F'])) + str(int(x['M']))

rfm['RFM_Segment'] = rfm.apply(join_rfm, axis=1)

rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()

,recency,frequency,monetary,age,R,F,M,RFM_Segment,RFM_Score
Customer ID,,,,,,,,,
12347.0,5,7,4060.40,370,5,5,5,555,15
12348.0,78,4,1186.68,361,2,4,4,244,10
12349.0,21,1,1353.80,21,4,1,4,414,9
12350.0,313,1,294.40,313,1,1,2,112,4
12352.0,39,7,1385.74,299,3,5,4,354,12


In [18]:
# Prepare the data for KMean Clustering
# Scale the recency, frequency and monetary values using MinMaxScaler

scaler = MinMaxScaler()

rfm_cluster = pd.DataFrame(scaler.fit_transform(rfm[["recency", "frequency", "monetary", "age"]]),
                            index=rfm.index, columns=["recency", "frequency", "monetary", "age"])

rfm_cluster.reset_index(inplace=True)

rfm_cluster.head()

,Customer ID,recency,frequency,monetary,age
0,12347.0,0.005362,0.029412,0.025525,0.983914
1,12348.0,0.201072,0.014706,0.007447,0.959786
2,12349.0,0.048257,0.000000,0.008498,0.048257
3,12350.0,0.831099,0.000000,0.001834,0.831099
4,12352.0,0.096515,0.029412,0.008699,0.793566
